In [458]:
import os, sys
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict
from functools import partial
from pathlib import Path
import json

In [459]:
output_dir = Path("capacity_data")
output_dir.mkdir(exist_ok=True, parents=True)

In [462]:
with open("db_config.json", 'r') as f:
    dbconfig = json.load(f)

In [464]:
import dolphindb as ddb 
s = ddb.session()
s.connect(dbconfig["DolphinURL"], dbconfig["DolphinPort"], dbconfig["DolphinAccount"], dbconfig["DolphinPassword"])

True

In [438]:

q1 = 'trade_table = loadTable("dfs://tick_trade", "trades")'
q2 = 'depth_table = loadTable("dfs://tick_depth", "depths")'
s.run(q1)
s.run(q2)

In [439]:
start_date, end_date = "2022-03-01", "2022-05-25"

In [446]:
symbols = {
    "ETH-USDC.UNS": ["ETH-USDT.CUP"],
    # "CAKE-BNB.PCS": ["CAKE-USDT.CUP"],
    # "CAKE-USDT.PCS": ["CAKE-USDT.CUP"],
    # "TRX-BUSD.PCS": ["TRX-USDT.CUP"],
    # "WOO-BNB.PCS": ["WOO-USDT.CUP"],

    # "ADA-BNB.PCS": ["ADA-USDT.CUP"],
    # "ADA-BUSD.PCS": ["ADA-USDT.CUP"],
    # "ALICE-BNB.PCS": ["ALICE-USDT.CUP"],
    # "BIFI-BNB.PCS": ["BIFI-USDT.CUP"],
    # "C98-BNB.PCS": ["C98-USDT.CUP"],
    # "CHR-BUSD.PCS": ["CHR-USDT.CUP"],
    # "DAI-BUSD.PCS": ["DAI-USDT.CUP"],
    # "DOGE-BNB.PCS": ["DOGE-USDT.CUP"],
    # "DOT-BNB.PCS": ["DOT-USDT.CUP"],
    # "ETH-BNB.PCS": ["ETH-USDT.CUP"],
    # "ETH-BUSD.PCS": ["ETH-USDT.CUP"],
    # "ETH-USDC.PCS": ["ETH-USDT.CUP"],
    # "LINK-BNB.PCS": ["LINK-USDT.CUP"],
    # "LTC-BNB.PCS": ["LTC-USDT.CUP"],
    # "MASK-BUSD.PCS": ["MASK-USDT.CUP"],
    # "MATIC-BNB.PCS": ["MATIC-USDT.CUP"],
    # "SPS-BNB.PCS": ["SPS-USDT.CUP"],
    # "TUSD-BUSD.PCS": ["TUSD-USDT.CUP"],
    # "UNI-BNB.PCS": ["UNI-USDT.CUP"],
    # "USDC-BNB.PCS": ["USDC-USDT.CUP"],
    # "USDC-BUSD.PCS": ["USDC-USDT.CUP"],
    # "XRP-BNB.PCS": ["XRP-USDT.PCS"],

    # "MMF-USDC.MMS": ["MMF-USDC.CUP"],
    # "MMF-USDT.MMS": ["MMF-USDC.CUP"],
    # "MMF-CRO.MMS":  ["MMF-USDC.CUP"],
    # "CRO-USDC.MMS": ["CRO-USDT.CUP"],
    # "CRO-USDT.MMS": ["CRO-USDT.CUP"],
    # "DAI-USDC.MMS": ["DAI-USDC.CUP"],
    # "CRO-WBTC.MMS": ["CRO-BTC.CUP"],
    # "ETH-WBTC.MMS": ["ETH-BTC.CUP"],

    # "CRO-USDC.VVS": ["CRO-USDT.CUP"],
    # "CRO-USDT.VVS": ["CRO-USDT.CUP"],
    # "CRO-WBTC.VVS": ["CRO-BTC.CUP"],
    # "ETH-CRO.VVS": ["ETH-CRO.CUP"],
    # "VVS-CRO.VVS": ["VVS-CRO.CUP"],
    # "USDC-USDT.VVS": ["USDC-USDT.CUP"],
    # "VVS-USDC.VVS": ["VVS-USDC.CUP"],
    # "VVS-USDT.VVS": ["VVS-USDC.CUP"],
    # "TONIC-CRO.VVS": ["TONIC-USDC.CUP"],
    # "TONIC-USDC.VVS": ["TONIC-USDC.CUP"],
    # "ETH-USDC.VVS": ["ETH-USDT.CUP"],
    # "ALI-CRO.VVS": ["ALI-USDT.CUP"],
    # "ATOM-CRO.VVS": ["ATOM-CRO.CUP"],
    # "BIFI-USDC.VVS": ["BIFI-USDT.CUP"],
    # "ELON-USDC.VVS": ["ELON-USDT.CUP"],
}

crossing_pairs = set([
    "USDCUSDT", 
    "DAIUSDT",
    "CROUSDT",
    "CROUSDC",
    "BNBUSDT",
    "BUSDUSDT",
])

wrapped = {
    "BTC": "WBTC",
    "ETH": "WETH",
    "CRO": "WCRO",
}

binance_symbols = set([
    "BUSD",
    "BNB",
])

coins = set([
    "ADA",
    "ALI",
    "ALICE",
    "ATOM",
    "AVAX",
    "BIFI",
    "BNB",
    "BTC",
    "C98",
    "CAKE",
    "CHR",
    "CRO",
    "DAI",
    "DOT",
    "DOGE",
    "ELON",
    "ETH",
    "LINK",
    "LTC",
    "MASK",
    "MATIC",
    "MMF",
    "SOL",
    "SPS",
    "TONIC",
    "TRX",
    "TSUD",
    "UNI",
    "USDC",
    "USDT",
    "VVS",
    "WBTC",
    "WCRO",
    "WETH",
    "WOO",
    "XRP",
])

In [447]:
gas_fees = {
    "UNS": 30,
    "MMS": 0.5,
    "VVS": 0.5,
    "PCS": 5,
}

bps_exchanges = set([
    "PCS"
])

delays = {
    "UNS": 15,
    "MMS": 6,
    "VVS": 6,
    "PCS": 3,
}

In [448]:
starts, ends = [], []
start = datetime.strptime(f"{start_date} 00:00:00", "%Y-%m-%d %H:%M:%S")
end = datetime.strptime(f"{end_date} 00:00:00", "%Y-%m-%d %H:%M:%S")
while start < end:
    starts.append(start.strftime("%Y.%m.%dT%H:%M:%S"))
    start += timedelta(days=1)
    ends.append(start.strftime("%Y.%m.%dT%H:%M:%S"))

In [449]:
def get_data_date(sym: str, hedge: str, side: str, delay: int, start: str, end: str, n_levels: int = 20) -> pd.DataFrame:
    price_type = 'a' if side == '1' else 'b'
    opposite_price_type = 'b' if side == '1' else 'a'
    volume_type = price_type + 'v'
    opposite_side = '-1' if side == '1' else '1'

    ticker, sym_ext = sym.split('.')
    sym_leg1, sym_leg2 = ticker.split('-')
    hedge_ticker, hedge_ext = hedge.split('.')
    hedge_leg1, hedge_leg2 = hedge_ticker.split('-')

    leg1_equal, leg2_equal = False, False
    if sym_leg1 == hedge_leg1 or ((sym_leg1 in wrapped) and (wrapped[sym_leg1] == hedge_leg1)) \
        or ((hedge_leg1 in wrapped) and (wrapped[hedge_leg1] == sym_leg1)):
        leg1_equal = True
    if sym_leg2 == hedge_leg2 or ((sym_leg2 in wrapped) and (wrapped[sym_leg2] == hedge_leg2)) \
        or ((hedge_leg2 in wrapped) and (wrapped[hedge_leg2] == sym_leg2)):
        leg2_equal = True


    if leg1_equal and leg2_equal:
        print(f"{sym} and {hedge} do not need a middle pair.")

        t1  = "t1 = select timestamp, sum(volume) as volume, wavg(price, volume) as vwap from trade_table"
        t1 += " where timestamp>={}, timestamp<{}, symbol='{}', side={} group by timestamp order by timestamp".format(
            start, end, sym.replace('-', ''), side)

        t2  = "t2 = select timestamp, " + price_type + f", {price_type}".join(map(str, list(range(1, n_levels + 1))))
        t2 += ', '
        t2 += volume_type + f", {volume_type}".join(map(str, list(range(1, n_levels + 1))))
        t2 += " from depth_table where timestamp>={}, timestamp<{}, symbol='{}', {}1>0 order by timestamp".format(
            start, end, hedge.replace('-', ''), price_type)

        t3  = "select * from wj(t1, t2, {}s:{}s, <[".format(delay, delay + 1)
        for level in range(1, n_levels + 1):
            t3 += f"first({price_type}{level}) as {price_type}{level}, first({volume_type}{level}) as {volume_type}{level}, "

        t3  = t3[:-2] + "]>, `timestamp)"

        s.run(t1)
        s.run(t2)
        df = s.run(t3)
        df["price"] = 1.0
        return df

    elif not leg1_equal and not leg2_equal:
        if sym_leg1 == hedge_leg2 and sym_leg2 == hedge_leg1:
            t1  = "t1 = select timestamp, sum(volume) as volume, wavg(price, volume) as vwap from trade_table"
            t1 += " where timestamp>={}, timestamp<{}, symbol='{}', side={} group by timestamp order by timestamp".format(
                start, end, sym.replace('-', ''), opposite_side)

            t2  = "t2 = select timestamp, " + price_type + f", {price_type}".join(map(str, list(range(1, n_levels + 1))))
            t2 += ', '
            t2 += volume_type + f", {volume_type}".join(map(str, list(range(1, n_levels + 1))))
            t2 += " from depth_table where timestamp>={}, timestamp<{}, symbol='{}', {}1>0 order by timestamp".format(
                start, end, hedge.replace('-', ''), price_type)

            t3  = "select * from wj(t1, t2, {}s:{}s, <[".format(delay, delay + 1)
            for level in range(1, n_levels + 1):
                t3 += f"first({price_type}{level}) as {price_type}{level}, first({volume_type}{level}) as {volume_type}{level}, "

            t3  = t3[:-2] + "]>, `timestamp)"

            # print(t1)
            # print(t2)
            # print(t3)

            print(f"{sym} and {hedge} are inversed.")
            s.run(t1)
            s.run(t2)
            df = s.run(t3)
            df["vwap"] = 1.0 / df["vwap"]
            df["price"] = 1.0
            return df
        else:
            print(f"Couldn't find arb between {sym} and {hedge}.")
            return pd.DataFrame()

    elif leg1_equal:
        print(f"{sym} and {hedge} are leg1_equal: {leg1_equal}.")
        
        cross, reverse_cross = None, None

        if sym_leg2 + hedge_leg2 in crossing_pairs:
            cross = sym_leg2 + hedge_leg2
        elif hedge_leg2 + sym_leg2 in crossing_pairs:
            reverse_cross = hedge_leg2 + sym_leg2

        t1  = "t1 = select timestamp, sum(volume) as volume, wavg(price, volume) as vwap from trade_table"
        t1 += " where timestamp>={}, timestamp<{}, symbol='{}', side={} group by timestamp order by timestamp".format(
            start, end, sym.replace('-', ''), side)

        t2  = "t2 = select timestamp, " + price_type + f", {price_type}".join(map(str, list(range(1, n_levels + 1))))
        t2 += ', '
        t2 += volume_type + f", {volume_type}".join(map(str, list(range(1, n_levels + 1))))
        t2 += " from depth_table where timestamp>={}, timestamp<{}, symbol='{}', {}1>0 order by timestamp".format(
            start, end, hedge.replace('-', ''), price_type)

        t3  = "select * from wj(t1, t2, {}s:{}s, <[".format(delay, delay + 1)
        for level in range(1, n_levels + 1):
            t3 += f"first({price_type}{level}) as {price_type}{level}, first({volume_type}{level}) as {volume_type}{level}, "

        t3  = t3[:-2] + "]>, `timestamp)"

        s.run(t1)

        if cross is None and reverse_cross is None:
            print(f"{sym} and {hedge} do not need a middle pair.")
        else:
            if cross:
                cross_ext = hedge_ext
                if cross.startswith("BNB") or cross.startswith("BUSD"):
                    cross_ext = "BNC"
                print(f"{sym} and {hedge} using crossing pair {cross}.{cross_ext}")
                s.run("cross_tbl = select timestamp, {} as price from depth_table where symbol='{}', timestamp>={}, timestamp<{};".format(
                    opposite_price_type+'1', cross + '.' + cross_ext,  start, end))
            else:
                reverse_cross_ext = hedge_ext
                if reverse_cross.startswith("BNB") or reverse_cross.startswith("BUSD"):
                    reverse_cross_ext = "BNC"
                print(f"{sym} and {hedge} using reverse crossing pair {reverse_cross}.{reverse_cross_ext}")
                s.run("cross_tbl = select timestamp, {} as price from depth_table where symbol='{}', timestamp>={}, timestamp<{};".format(
                    f"1.0 / {price_type}1", reverse_cross + '.' + reverse_cross_ext, start, end))
            s.run("t1 = wj(t1, cross_tbl, 0s:120s, <first(price) as price>, `timestamp)");

        s.run(t2)
        df = s.run(t3)

        return df
    
    else:
        print(f"{sym} and {hedge} are leg2_equal: {leg2_equal}.")
        return pd.DataFrame()


def get_data(sym: str, hedge: str, side: str, delay: int, starts: List, ends: List) -> pd.DataFrame:
    n = len(starts)
    if not n > 0: return pd.DataFrame()

    res = []
    for i in range(n):
        print(f"Loading data {sym} from {starts[i]}")
        df_tmp = get_data_date(sym, hedge, side, delay, starts[i], ends[i])
        if not df_tmp.empty:
            res.append(df_tmp)

    if len(res) > 0:
        return pd.concat(res)

    return pd.DataFrame()

In [450]:
def buy_match(exch, fee, n_levels, x):
    res, pnl = 0, 0
    tot = x["volume"]
    for i in range(1, n_levels + 1):
        price = x["vwap"] * x["price"]
        if not x[f"a{i}"] > 0 or price < x[f"a{i}"] or tot <= 0:
            break

        vol = min(tot, x[f"av{i}"])
        res += vol * x[f"a{i}"]
        pnl += vol * (price - x[f"a{i}"])
        tot -= vol

    if exch in bps_exchanges:
        fee = res * fee / 1e4

    if pnl < fee:
        return 0, 0, 0

    pnl -= fee 

    return res, pnl, fee

def sell_match(exch, fee, n_levels, x):
    res, pnl = 0, 0
    tot = x["volume"]
    for i in range(1, n_levels + 1):
        price = x["vwap"] * x["price"]
        if not x[f"b{i}"] > 0 or price > x[f"b{i}"] or tot <= 0:
            break

        vol = min(tot, x[f"bv{i}"])
        res += vol * x[f"b{i}"]
        pnl += vol * (x[f"b{i}"] - price)
        tot -= vol

    if exch in bps_exchanges:
        fee = res * fee / 1e4

    if pnl < fee:
        return 0, 0, 0

    pnl -= fee

    return res, pnl, fee


In [451]:
results = []
trades_L = []
trades_S = []
for sym, hedges in symbols.items():
    _, exch = sym.split('.')
    gas = gas_fees[exch]
    delay = delays[exch]
    print(f"delay: {delay}, symbol: {sym}")
    for hedge in hedges:
        df_L = get_data(sym, hedge, '1', delay, starts, ends)

        if not df_L.empty:
            df_L = df_L.loc[(df_L["vwap"] > 0) & (df_L["volume"] > 0) & (df_L["price"] > 0)]
            if not df_L.empty:
                buy_func = partial(buy_match, exch, gas, 20)
                buys = df_L.apply(buy_func, axis=1)
                df_L["potential_match"] = buys.str[0]
                df_L["potential_pnl"] = buys.str[1]
                df_L["fee"] = buys.str[2]
                df_L["timestamp"] = pd.to_datetime(df_L["timestamp"])
                df_L.set_index("timestamp", inplace=True)
                df_L["symbol"] = sym.replace('-', '')
                df_L["hedge"] = hedge.replace('-', '')
                df_L["delay"] = delay
                trades_L.append(df_L)

        df_S = get_data(sym, hedge, '-1', delay, starts, ends)

        if not df_S.empty:
            df_S = df_S.loc[(df_S["vwap"] > 0) & (df_S["volume"] > 0) & (df_S["price"] > 0)]
            if not df_S.empty:
                sell_func = partial(sell_match, exch, gas, 20)
                sells = df_S.apply(sell_func, axis=1)

                df_S["potential_match"] = sells.str[0]
                df_S["potential_pnl"] = sells.str[1]
                df_S["fee"] = sells.str[2]
                df_S["timestamp"] = pd.to_datetime(df_S["timestamp"])
                df_S.set_index("timestamp", inplace=True)
                df_S["symbol"] = sym.replace('-', '')
                df_S["hedge"] = hedge.replace('-', '')
                df_S["delay"] = delay
                trades_S.append(df_S)

        d = {}
        if len(df_L) > 0:
            d["LongMatch"] = df_L["potential_match"].resample("1D").sum()
            d["LongPnL"] = df_L["potential_pnl"].resample("1D").sum()
            d["LongFee"] = df_L["fee"].resample("1D").sum()

        if len(df_S) > 0:
            d["ShortMatch"] = df_S["potential_match"].resample("1D").sum()
            d["ShortPnL"] = df_S["potential_pnl"].resample("1D").sum()
            d["ShortFee"] = df_S["fee"].resample("1D").sum()

        df = pd.DataFrame(d)

        df["symbol"] = sym.replace('-', '')
        df["hedge"] = hedge.replace('-', '')
        df["delay"] = delay

        results.append(df)

        output_file = output_dir / "{}-{}.csv".format(sym.replace('-', ''), hedge.replace('-', ''))
        df.to_csv(output_file, index=True)

df = pd.concat(results)

if len(trades_L) > 0: 
    df_L_tot = pd.concat(trades_L)
    df_L_tot.to_csv(output_dir / "trades_L.csv")
if len(trades_S) > 0: 
    df_S_tot = pd.concat(trades_S)
    df_S_tot.to_csv(output_dir / "trades_S.csv")

delay: 15, symbol: ETH-USDC.UNS
Loading data ETH-USDC.UNS from 2022.03.01T00:00:00
ETH-USDC.UNS and ETH-USDT.CUP are leg1_equal: True.
ETH-USDC.UNS and ETH-USDT.CUP using crossing pair USDCUSDT.CUP
Loading data ETH-USDC.UNS from 2022.03.02T00:00:00
ETH-USDC.UNS and ETH-USDT.CUP are leg1_equal: True.
ETH-USDC.UNS and ETH-USDT.CUP using crossing pair USDCUSDT.CUP
Loading data ETH-USDC.UNS from 2022.03.03T00:00:00
ETH-USDC.UNS and ETH-USDT.CUP are leg1_equal: True.
ETH-USDC.UNS and ETH-USDT.CUP using crossing pair USDCUSDT.CUP
Loading data ETH-USDC.UNS from 2022.03.04T00:00:00
ETH-USDC.UNS and ETH-USDT.CUP are leg1_equal: True.
ETH-USDC.UNS and ETH-USDT.CUP using crossing pair USDCUSDT.CUP
Loading data ETH-USDC.UNS from 2022.03.05T00:00:00
ETH-USDC.UNS and ETH-USDT.CUP are leg1_equal: True.
ETH-USDC.UNS and ETH-USDT.CUP using crossing pair USDCUSDT.CUP
Loading data ETH-USDC.UNS from 2022.03.06T00:00:00
ETH-USDC.UNS and ETH-USDT.CUP are leg1_equal: True.
ETH-USDC.UNS and ETH-USDT.CUP using

In [452]:
df_L

,volume,vwap,price,a1,av1,a2,av2,a3,av3,a4,...,a19,av19,a20,av20,potential_match,potential_pnl,fee,symbol,hedge,delay
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-03-02 00:32:31,299.408528,2939.418278,0.9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-03-02 00:33:29,6.485058,2941.360458,0.9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-03-02 00:33:54,1.632541,2937.996551,0.9995,2930.90,0.12621,2930.91,0.11951,2930.99,0.18649,2931.02,...,2932.18,0.00034,2932.20,0.06842,0.000000,0.000000,0,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-03-02 00:34:40,11.774316,2933.154963,0.9995,2926.42,0.06732,2926.48,0.51244,2926.83,0.68346,2926.86,...,2928.06,0.00100,2928.12,0.00040,34462.779594,25.845337,30,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-03-02 00:35:05,10.260470,2931.225647,0.9995,2929.17,0.05463,2929.27,0.68292,2929.40,0.00200,2929.43,...,2930.01,0.33509,2930.06,0.51189,0.000000,0.000000,0,ETHUSDC.UNS,ETHUSDT.CUP,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-24 23:57:28,5.546223,1978.758856,1.0011,1982.18,1.26135,1982.25,1.45568,1982.37,0.25000,1982.38,...,1982.78,0.50040,1982.80,1.17211,0.000000,0.000000,0,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-05-24 23:58:21,10.107162,1978.794851,1.0011,1978.73,2.53910,1978.75,1.00674,1978.76,2.52683,1978.79,...,1979.32,1.57210,1979.40,1.24767,0.000000,0.000000,0,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-05-24 23:58:30,0.515330,1980.156928,1.0011,1978.37,0.75923,1978.45,3.43367,1978.46,2.52722,1978.47,...,1979.01,2.32913,1979.09,0.50000,0.000000,0.000000,0,ETHUSDC.UNS,ETHUSDT.CUP,15


In [453]:
df2 = df.reset_index()

In [454]:
df2

,timestamp,LongMatch,LongPnL,LongFee,ShortMatch,ShortPnL,ShortFee,symbol,hedge,delay
0,2022-03-02,2.387082e+07,19496.283614,6480,3.357524e+07,27561.393005,8490,ETHUSDC.UNS,ETHUSDT.CUP,15
1,2022-03-03,2.789054e+06,6804.172060,3090,2.780745e+06,4717.677550,3150,ETHUSDC.UNS,ETHUSDT.CUP,15
2,2022-03-04,3.466832e+07,28855.626132,10380,3.697246e+07,31360.745281,11370,ETHUSDC.UNS,ETHUSDT.CUP,15
3,2022-03-05,1.830370e+07,12835.533867,6300,1.613984e+07,11538.695611,6210,ETHUSDC.UNS,ETHUSDT.CUP,15
4,2022-03-06,1.737964e+07,13768.773930,5880,1.840705e+07,14646.158983,7080,ETHUSDC.UNS,ETHUSDT.CUP,15
...,...,...,...,...,...,...,...,...,...,...
79,2022-05-20,2.840287e+07,22270.002311,7560,4.004679e+07,32937.935364,10260,ETHUSDC.UNS,ETHUSDT.CUP,15
80,2022-05-21,6.680813e+06,4038.629994,2490,8.903468e+06,4513.885599,3120,ETHUSDC.UNS,ETHUSDT.CUP,15
81,2022-05-22,1.349174e+07,9558.267968,4410,1.627072e+07,13323.161454,5370,ETHUSDC.UNS,ETHUSDT.CUP,15
82,2022-05-23,9.464138e+05,2949.776279,1290,9.798967e+05,1319.678599,1950,ETHUSDC.UNS,ETHUSDT.CUP,15


In [455]:
df_sum = pd.DataFrame({
    "LongMatch": df2.groupby(["symbol", "hedge"])["LongMatch"].sum(),
    "ShortMatch": df2.groupby(["symbol", "hedge"])["ShortMatch"].sum(),
    "LongPnL": df2.groupby(["symbol", "hedge"])["LongPnL"].sum(),
    "LongFee": df2.groupby(["symbol", "hedge"])["LongFee"].sum(),
    "ShortPnL": df2.groupby(["symbol", "hedge"])["ShortPnL"].sum(),
    "StartDate": df2.groupby(["symbol", "hedge"])["timestamp"].min(),
    "ShortFee": df2.groupby(["symbol", "hedge"])["ShortFee"].sum(),
})

In [456]:
df_sum["PnL"] = df_sum["LongPnL"] + df_sum["ShortPnL"]
df_sum["Fee"] = df_sum["LongFee"] + df_sum["ShortFee"]
df_sum["NTL"] = df_sum["LongMatch"] + df_sum["ShortMatch"]
df_sum["Margin"] = df_sum["PnL"].div(df_sum["NTL"])

In [457]:
df_sum

,,LongMatch,ShortMatch,LongPnL,LongFee,ShortPnL,StartDate,ShortFee,PnL,Fee,NTL,Margin
symbol,hedge,,,,,,,,,,,
ETHUSDC.UNS,ETHUSDT.CUP,1.893599e+09,2.258477e+09,1.817537e+06,627330,2.715126e+06,2022-03-02,742920,4.532663e+06,1370250,4.152076e+09,0.001092


In [434]:
df_sum.to_csv(output_dir / "capacity_PCS_fixed.csv", index=True)

In [415]:
df_L.loc[df_L["potential_match"] > df_L["volume"].mul(df_L["vwap"])]

,volume,vwap,price,a1,av1,a2,av2,a3,av3,a4,...,a19,av19,a20,av20,potential_match,potential_pnl,fee,symbol,hedge,delay
timestamp,,,,,,,,,,,,,,,,,,,,,


In [416]:

df_S.loc[df_S["potential_match"] > df_S["volume"].mul(df_S["vwap"])]

,volume,vwap,price,b1,bv1,b2,bv2,b3,bv3,b4,...,b19,bv19,b20,bv20,potential_match,potential_pnl,fee,symbol,hedge,delay
timestamp,,,,,,,,,,,,,,,,,,,,,


In [112]:
17522.8*0.186

3259.2408

In [113]:
4026*0.18

724.68